In [1]:
using PyPlot

INFO: Loading help data...


In [7]:
# plot in an external window since it doesn't work yet in IJulia
pygui(true)
# tell PyPlot that the plot is interactive
PyPlot.ion()
# . . . and that previous plots are overwritten
#PyPlot.hold(false)

fig = plt.figure()
ax=fig[:add_subplot](111)

x=[1:1:100]
y=[rand(100)]

li=ax[:plot](x, y)[1]

fig[:canvas][:draw]()
plt.show()

y2=[rand(500,100)]

@time for i=1:500
    li[:set_ydata](y2[i,:])
    fig[:canvas][:blit](ax[:bbox])
    #fig[:canvas][:draw]()
end

elapsed time: 1.152237795 seconds (3663560 bytes allocated)


In [1]:
using ExtractSpikes

In [9]:
s=SpikeDetection()
c=Cluster()
be=ones(Int64,20000)
prepareDetection(s,be)
@time detectSpikes(s,c,be,10.0,10.0)

elapsed time: 0.00037952 seconds (448 bytes allocated)


([0],[0])